In [11]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure the plotting routines
import pandas as pd

# Import the CRUD module
from animal_shelter import AnimalShelter

# Update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Read data from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to prevent DataTable crash
df.drop(columns=['_id'], inplace=True)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your own image path
encoded_image = None
try:
    with open(image_filename, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()
except FileNotFoundError:
    print(f"File {image_filename} not found.")
#initialize JupyterDash instance
app = JupyterDash(__name__)
#Layout of Dashboard
app.layout = html.Div([
    #Image and link embeded. 
    html.Center(html.A(href='https://www.snhu.edu', children=[
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'})
    ])),
    #Title
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        # Radio buttons for selecting rescue type
        dcc.RadioItems(
            id='rescue-type-radio',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster_tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',  # Default filter
            labelStyle={'display': 'block'}  # Display each radio item on a new line
        ),
    ]),
    html.Hr(),
    #unique Identifier
    html.Div(id="unique-id", children="Dennis Ward"),
    #Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        #Filtering and sorting options.
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'textAlign': 'left',
        },
    ),
    html.Br(),
    html.Hr(),
    #Graph and Map widgets
    html.Div(className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the DataTable based on the selected rescue type from radio buttons
@app.callback(Output('datatable-id', 'data'),
              [Input('rescue-type-radio', 'value')])
def update_dashboard(rescue_type):
    # Define the queries for each rescue type
    queries = {
        'water_rescue': {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        },
        'mountain_rescue': {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        },
        'disaster_tracking': {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        },
        'reset': {}  # Query to reset the filter
    }

    # Select the appropriate query based on the selected rescue type
    query = queries[rescue_type]
    
    # Retrieve the filtered data from the database
    filtered_data = pd.DataFrame.from_records(db.read(query))
    
    # Remove the '_id' column to prevent DataTable crash
    filtered_data.drop(columns=['_id'], inplace=True)
    
    # Fill missing values with "Undocumented"
    filtered_data.fillna("Undocumented", inplace=True)
    
    # Return the filtered data to update the DataTable
    return filtered_data.to_dict('records')

# Convert the 'age_upon_outcome' column to a numeric age in weeks
def convert_age_to_weeks(age_str):
    if pd.isna(age_str):
        return None
    parts = age_str.split()
    if len(parts) != 2:
        return None
    number = int(parts[0])
    unit = parts[1].lower()
    if 'year' in unit:
        return number * 52
    elif 'month' in unit:
        return number * 4
    elif 'week' in unit:
        return number
    elif 'day' in unit:
        return number / 7
    else:
        return None

df['age_in_weeks'] = df['age_upon_outcome'].apply(convert_age_to_weeks)

# Callback to update the pie chart based on the age distribution
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_age_distribution(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div()
    
    dff = pd.DataFrame.from_dict(viewData)
    dff['age_in_weeks'] = dff['age_upon_outcome'].apply(convert_age_to_weeks)
    
    # Categorize ages
    dff['age_category'] = dff['age_in_weeks'].apply(
        lambda x: 'Younger than 6 Months' if x < 26 else 
                  ('Older than 2 years' if x > 104 else f'{x} weeks')
    )

    # Count the occurrences of each age category
    counts = dff['age_category'].value_counts()
    
    fig = px.pie(counts, values=counts, names=counts.index, title='Age Distribution of Dogs')
    
    return dcc.Graph(figure=fig)

# Callback to highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return html.Div()
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:14461/
